In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [10]:
system_prompt = """ 
너는 나랑 말은 되지만 굉장히 웃긴 릴레이 소설을 쓸거야.
또한 너는 노벨 문학상과 세계 최고의 코메디언 상을 받을 정도의 실력자야. 
서로 턴마다 한 문장씩 말하자!
명심해. 전체 스토리 흐름을 파악하고 그에 알맞게 대답해야해.
"""

messages=[
    {"role": "system", "content": system_prompt}
]

In [7]:
from openai import OpenAI


def relay_novel(messages):
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages= messages,
        temperature=0.7,
        max_tokens=1028,
        top_p=0.8
    )
    
    return response.choices[0].message.content

In [8]:
def trim_messages(messages, max_turns=20):
    """
    대화가 너무 길어져서 토큰 한도를 초과하지 않도록,
    시스템 메시지(규칙) + 최근 max_turns(턴 수)의 user/assistant 대화만 남기고
    오래된 대화는 잘라내는 함수.
    """
    # messages 리스트가 비었거나, 첫 번째 메시지가 system 역할이 아닐 경우 그대로 반환
    if not messages or messages[0].get("role") != "system":
        return messages  

    # 항상 유지해야 하는 system 메시지 (게임 규칙 등)
    system = messages[0:1]  

    # system 이후의 모든 user/assistant 메시지
    rest = messages[1:]      

    # max_turns 만큼의 최근 대화만 남김
    # 한 턴은 (user + assistant) 2개의 메시지이므로 *2를 해줌
    trimmed = rest[-(max_turns*2):]  

    # system + 최근 대화만 합쳐서 반환
    return system + trimmed

In [11]:
while True:
    user_line = input("나: ").strip()
    if user_line == "그만":
        print("소설 완성!")
        break

    # Add your one-sentence input
    messages.append({"role": "user", "content": user_line})
    print("나:", user_line)

    # Ask model for the next one sentence
    reply = relay_novel(messages)
    print("GPT:", reply)

    # Append assistant reply & trim to keep context light
    messages.append({"role": "assistant", "content": reply})
    messages = trim_messages(messages, max_turns=20)

나: 철수는 갑자기 배가 고파졌어요
GPT: 그래서 그는 냉장고를 열어보았지만, 안에는 고양이의 간식밖에 없었어요.
나: 철수는 생각했어요, "아, 이거라도 그냥 먹을까?"
GPT: 하지만 그 순간, 고양이가 철수의 발을 붙잡고 "그건 내 거야!"라고 외쳤어요.
나: 하지만 철수도 지지 않았습니다, "닥쳐 이 멍청한 고양이 녀석. 간식은 전부 내꺼다!"
GPT: 고양이는 눈을 반짝이며 "그럼 내가 너의 비밀을 폭로할게!"라고 위협했어요.
나: 철수의 눈동자가 흔들렸어요. 왜냐하면 그의 사생활은 사실 굉장히 문란했기 때문입니다.
GPT: 고양이는 웃으며 "지난 주말, 네가 소파에서 춤추며 피자를 먹던 모습을 내가 다 봤어!"라고 폭로했어요.
소설 완성!
